In [1]:
import time
import torch
import torch.nn as nn
import numpy as np
import random
from torch import optim
import matplotlib.pyplot as plt
from typing import List
from utils import *
from torch.utils.data import Dataset, DataLoader, RandomSampler
import tqdm
from sklearn.decomposition import PCA
from scipy.stats import ttest_ind
from bus_transformer import *
from datasets import load_dataset
from transformers import AutoTokenizer

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

cuda


- limit sequences to 128
- limit tasks to sentence classification
- use single sequence training without NSP
- 


In [2]:
data = load_dataset('Salesforce/wikitext', 'wikitext-103-v1')

In [3]:
data.column_names

{'test': ['text'], 'train': ['text'], 'validation': ['text']}

In [4]:
data['train'].num_rows

1801350

In [5]:
data['test'].num_rows

4358

In [6]:
data['validation'].num_rows

3760

In [7]:
train = data['train']
validation = data['validation']
test = data['test']

In [8]:
seq_len = 128

In [9]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')

/home/brian/anaconda3/envs/NLP/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
def remove_empty(examples):
    ret = {}
    ret['text'] = []
    for ex in examples['text']:
        if len(ex) > 2:
            ret['text'].append(ex)

    return ret


In [11]:
def tokenize_func(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

In [12]:
data_train_remove_empty = train.map(remove_empty, batched=True)

In [13]:
data_valid = validation.map(remove_empty, batched=True)
data_test = test.map(remove_empty, batched=True)

In [14]:
data_train_remove_empty['text'][0:10]

[' = Valkyria Chronicles III = \n',
 ' Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . \n',
 " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for serie

In [15]:
tk_train = data_train_remove_empty.map(tokenize_func, batched=True)

In [16]:
tk_valid = data_valid.map(tokenize_func, batched=True)

In [17]:
tk_test = data_test.map(tokenize_func, batched=True)

Map:   0%|          | 0/2891 [00:00<?, ? examples/s]

In [18]:
tk_train

Dataset({
    features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1165029
})

In [19]:
model = Decoder(num_blocks=3, d_model=128, d_internal=64, vocab_size=30522, num_heads=3, d_hidden=2048, final_dmodel=128)

In [20]:
torch.tensor(tk_train[100]['input_ids']).shape

torch.Size([128])

In [21]:
from importlib import reload
import bus_transformer as bus

In [26]:
reload(bus)
import bus_transformer as bus
model = bus.Decoder(num_blocks=3, d_model=128, d_internal=64, vocab_size=30522, num_heads=3, d_hidden=2048, final_dmodel=128)
model(torch.tensor(tk_train[100]['input_ids'])).shape

torch.Size([1, 128, 30522])

In [28]:
py = model(torch.tensor(tk_train[101]['input_ids']))

In [32]:
import tensorboard_data_server

In [ ]:
def training_loop(model, data, dev, epochs):
    optimizer = optim.AdamW(model.parameters(), lr=1e-5)
    loss = torch.nn.CrossEntropyLoss()
    for t in range(epochs):
        for i, d in enumerate(data):
            y = d
            py = model(d)
            l = loss(py, d)
            model.zero_grad()
            loss.backward()
            optimizer.step()

    return model


In [29]:
loss = torch.nn.CrossEntropyLoss()
loss(py, torch.tensor(tk_train[101]['input_ids']))

ValueError: Expected input batch_size (1) to match target batch_size (128).

In [30]:
tk_train[101]['text']

" Cicely Mary Barker ( 28 June 1895 – 16 February 1973 ) was an English illustrator best known for a series of fantasy illustrations depicting fairies and flowers . Barker 's art education began in girlhood with correspondence courses and instruction at the Croydon School of Art . Her earliest professional work included greeting cards and juvenile magazine illustrations , and her first book , Flower Fairies of the Spring , was published in 1923 . Similar books were published in the following decades . \n"

In [31]:
tk_train[101]['input_ids']

[101,
 25022,
 29109,
 2100,
 2984,
 12852,
 1006,
 2654,
 2238,
 6301,
 1516,
 2385,
 2337,
 3381,
 1007,
 2001,
 2019,
 2394,
 13825,
 2190,
 2124,
 2005,
 1037,
 2186,
 1997,
 5913,
 11249,
 10775,
 20182,
 1998,
 4870,
 1012,
 12852,
 1005,
 1055,
 2396,
 2495,
 2211,
 1999,
 2611,
 9021,
 2007,
 11061,
 5352,
 1998,
 7899,
 2012,
 1996,
 21838,
 2082,
 1997,
 2396,
 1012,
 2014,
 5700,
 2658,
 2147,
 2443,
 14806,
 5329,
 1998,
 11799,
 2932,
 11249,
 1010,
 1998,
 2014,
 2034,
 2338,
 1010,
 6546,
 20182,
 1997,
 1996,
 3500,
 1010,
 2001,
 2405,
 1999,
 4927,
 1012,
 2714,
 2808,
 2020,
 2405,
 1999,
 1996,
 2206,
 5109,
 1012,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]